In [ ]:
# Install necessary packages
!pip install fastapi uvicorn pydantic diffusers transformers accelerate
!pip install pyngrok

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from diffusers import StableDiffusionPipeline
import torch
import base64
from io import BytesIO
import nest_asyncio
from pyngrok import ngrok
import uvicorn

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
app = FastAPI()

In [ ]:
# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

In [ ]:
# Load your model
model_id = "sayeedahmed/Derma-finetune-Diffusion-Model"
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, use_safetensors=True)
pipe = pipe.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

safety_checker/config.json:   0%|          | 0.00/786 [00:00<?, ?B/s]

tokenizer/special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

...

tokenizer/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/816 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
class PromptInput(BaseModel):
    color: str
    rash_type: str
    body_part: str

def create_prompt(color, rash_type, body_part):
    return (f"Create a highly detailed and realistic image showing the {body_part} of a person with {color} skin. "
            f"The {body_part} should display a typical {rash_type} infection, characterized by a clearly visible rash.")


In [ ]:
@app.post("/generate_image")
async def generate_image(prompt_input: PromptInput):
    # Create the prompt string
    prompt_text = create_prompt(prompt_input.color, prompt_input.rash_type, prompt_input.body_part)
    with torch.inference_mode():
        image = pipe(prompt_text, num_inference_steps=30).images[0]
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode()
    return {"image": img_str}

In [ ]:
# Set up and run the server
if __name__ == "__main__":
    nest_asyncio.apply()
    ngrok_auth_token = input("Please enter your ngrok auth token: ")
    ngrok.set_auth_token(ngrok_auth_token)

    try:
        ngrok_tunnel = ngrok.connect(8000)
        print('Public URL:', ngrok_tunnel.public_url)
        uvicorn.run(app, host="0.0.0.0", port=8000)
    except Exception as e:
        print(f"An error occurred: {str(e)}")

INFO:     Started server process [1768]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: https://bdf6-34-19-97-136.ngrok-free.app


  0%|          | 0/30 [00:00<?, ?it/s]

INFO:     2601:19b:500:8c10:1a4:f537:3f7d:3322:0 - "POST /generate_image HTTP/1.1" 200 OK


  0%|          | 0/30 [00:00<?, ?it/s]

...

INFO:     2601:19b:500:8c10:fd7a:6434:e531:ca12:0 - "POST /generate_image HTTP/1.1" 200 OK


  0%|          | 0/30 [00:00<?, ?it/s]

INFO:     2601:19b:500:8c10:fd7a:6434:e531:ca12:0 - "POST /generate_image HTTP/1.1" 200 OK


  0%|          | 0/30 [00:00<?, ?it/s]

INFO:     2601:19b:500:8c10:fd7a:6434:e531:ca12:0 - "POST /generate_image HTTP/1.1" 200 OK
